# 🔍 Complaila Database Explorer

This notebook allows you to interactively browse the Complaila SQLite database using the project's `SQLiteClient`.

In [1]:
import sys
import os
from pathlib import Path
from IPython.display import display, Markdown

# Add project root to path
project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.database.sqlite_client import SQLiteClient

client = SQLiteClient()
print("✅ Connected to database at:", client.db_path)

✅ Connected to database at: /home/alex/dev/complaila/data/complaila.db


## 1. List All Tables

In [2]:
def list_tables():
    cursor = client.conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall() if not row[0].startswith('sqlite_')]
    
    md = "### Tables in Database\n\n"
    md += "| Table Name | Row Count |\n"
    md += "| :--- | :--- |\n"
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        md += f"| {table} | {count} |\n"
        
    display(Markdown(md))

list_tables()

### Tables in Database

| Table Name | Row Count |
| :--- | :--- |
| document_chunks | 74 |
| vec_document_chunks | 74 |
| vec_document_chunks_info | 4 |
| vec_document_chunks_chunks | 1 |
| vec_document_chunks_rowids | 74 |
| vec_document_chunks_vector_chunks00 | 1 |
| questionnaires | 0 |
| questions | 0 |
| run_configurations | 0 |
| runs | 0 |
| answers | 0 |
| citations | 0 |
| retrieved_chunks | 0 |


## 2. Browse Table Content
Change the `table_name` variable to browse different tables.

In [5]:
def show_table(table_name, limit=5):
    cursor = client.conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
    rows = cursor.fetchall()
    
    if not rows:
        print(f"Table '{table_name}' is empty.")
        return
        
    headers = rows[0].keys()
    
    md = f"### Top {limit} rows from `{table_name}`\n\n"
    md += "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    
    for row in rows:
        # Clean up cell content for markdown
        cells = [str(row[h]).replace('\n', '<br>')[:100] + ('...' if len(str(row[h])) > 100 else '') for h in headers]
        md += "| " + " | ".join(cells) + " |\n"
        
    display(Markdown(md))

show_table("questions")

### Top 5 rows from `questions`

| id | questionnaire_id | question_id | text | section | sequence |
| --- | --- | --- | --- | --- | --- |
| sample_questionnaire:Q1.1 | sample_questionnaire | Q1.1 | What security certifications does your organization hold? | Section 1: Security & Compliance Certifications | 1 |
| sample_questionnaire:Q1.2 | sample_questionnaire | Q1.2 | Are you SOC 2 compliant? If yes, which Trust Services Criteria are covered and when was your last au... | Section 1: Security & Compliance Certifications | 2 |
| sample_questionnaire:Q1.3 | sample_questionnaire | Q1.3 | Do you maintain ISO 27001 certification? Please provide certification details. | Section 1: Security & Compliance Certifications | 3 |
| sample_questionnaire:Q1.4 | sample_questionnaire | Q1.4 | What compliance frameworks do you adhere to (GDPR, CCPA, etc.)? | Section 1: Security & Compliance Certifications | 4 |
| sample_questionnaire:Q2.1 | sample_questionnaire | Q2.1 | What technology stack is your platform built on? | Section 2: Technical Infrastructure | 5 |


## 3. Custom SQL Query

In [ ]:
query = """
SELECT 
    q.text as question,
    a.answer_text,
    a.is_success
FROM questions q
JOIN answers a ON q.id = a.question_id
LIMIT 3
"""

cursor = client.conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()

if rows:
    headers = rows[0].keys()
    md = "### Query Results\n\n"
    md += "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    for row in rows:
        cells = [str(row[h]).replace('\n', '<br>')[:100] for h in headers]
        md += "| " + " | ".join(cells) + " |\n"
    display(Markdown(md))
